In [1]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Model and offload folder paths
model_path = "/content/drive/MyDrive/phi2-full-merged-model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Function to normalize user input
def normalize_input(user_input):
    instruction = (
        "Instruct: You are a helpful assistant that normalizes and corrects user-submitted personal details "
        "like name, date of birth, email, and address.\n"
    )
    prompt = instruction + f"Input: {user_input}\nOutput:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.95,
            top_k=50,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded[len(prompt):].strip()

    return response

# Gradio UI
iface = gr.Interface(
    fn=normalize_input,
    inputs=gr.Textbox(lines=5, placeholder="Enter personal details here..."),
    outputs="text",
    title="Phi-2 Personal Info Normalizer",
    description="Fine-tuned Phi-2 model that standardizes personal data like Name, DOB, Email, and Address."
)

iface.launch()


ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.